# DDPG (Deep Deterministic Policy Gradient)

# 03. DDPG

[T. P. Lillicrap et al., "Continuous control with deep reinforcement learning." arXiv preprint arXiv:1509.02971, 2015.](https://arxiv.org/pdf/1509.02971.pdf)

<img src="https://enfow.github.io/image/paper-review/ddpg_algorithm.png" width=600 />
<img src="https://miro.medium.com/max/1200/1*-87grz5iUZK4i7NCH1ldbw.png" width=300 />

Deep Q Network(DQN)([Mnih et al., 2013;2015](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf)) 알고리즘은 강화 학습과 Deep Learning의 발전을 결합합니다. 그러나 DQN은 고차원 observation 공간의 문제를 해결하지만 greedy 정책을 사용하기 때문에 이산 및 저차원 action space만 처리할 수 있습니다. 고차원적이고 연속적인 action space에서 학습하기 위해 저자는 actor critic 접근 방식과 최근 DQN 성공의 통찰력을 결합합니다. Deep DPG(DDPG)는 DPG(Deterministic Policy Gradient) 알고리즘을 기반으로 합니다(Silver et al., 2014).

### Deterministic policy gradient
DPG 알고리즘은 state를 특정 action에 deterministic하게 매핑하여 현재 정책을 지정하는 매개변수화된 actor 함수 $\mu(s|\theta^{\mu})$를 유지 관리합니다. critic $Q(s, a)$는 Q-learning에서와 같이 Bellman 방정식을 사용하여 학습됩니다. actor는 액터 매개변수와 관련하여 시작 분포 $J$의 기대 이득(expected return)에 chain rule을 적용하여 업데이트됩니다.

$$
\begin{align*}
\nabla_{\theta^{\mu}}J &\approx E_{s_t\sim\rho^\beta} [\nabla_{\theta^{\mu}} Q(s,a|\theta^Q)|_{s=s_t, a=\mu(s_t|\theta^\mu)}] \\|
&= E_{s_t\sim\rho^\beta} [\nabla_{a} Q(s,a|\theta^Q)|_{s=s_t, a=\mu(s_t)} \nabla_{\theta^{\mu}} \mu(s|\theta^\mu)|_{s=s_t}]
\end{align*}
$$

### Replay buffer
강화 학습에 신경망을 사용할 때의 한 가지 문제는 대부분의 최적화 알고리즘이 샘플이 독립적이고 동일하게 분포(iid)되어 있다고 가정한다는 것입니다. 샘플이 환경에서 순차적으로 탐색하여 생성되면 이 가정은 더 이상 유효하지 않습니다.  **재생 버퍼**를 사용하여 이러한 문제를 해결합니다. 각 시간 단계에서 actor와 crictor는 버퍼에서 균일하게 미니배치를 샘플링하여 업데이트됩니다. 일련의 **uncorrelated** transition에서 학습의 이점을 얻을 수 있습니다.

### Soft update target network
업데이트되는 네트워크 $(Q(s,a|\theta^Q)$는 타겟 값 계산에도 사용되기 때문에 Q 업데이트는 발산하기 쉽습니다. 이를 피하기 위해  **target network**를 사용합니다. DQN과 비슷하지만 actor-critic 용으로 수정되고 ***soft target update**를 사용합니다. 타겟 네트워크는 대상 값을 계산하는 데 사용되는 actor 및 critic 네트워크, $Q'(s,a|\theta^{Q'})$ and $\mu'(s|\theta^{\mu`})$를 복사하여 생성됩니다.   그런 다음 이러한 대상 네트워크의 가중치는 학습된 네트워크를 천천히 추적하도록 하여 업데이트됩니다.

$$
\theta' \leftarrow \tau \theta + (1 - \tau)\theta' \ \ \ {with} \ \tau \ll 1.
$$

학습의 안정성을 크게 향상시킵니다.

### Exploration for continuous action space
DDPG와 같은 off-policy 알고리즘의 장점은 학습 알고리즘과 독립적으로 탐색 문제를 처리할 수 있다는 것입니다.  noise process $\mathcal{N}$에서 샘플링된 noise를 액터 정책에 추가하여 탐색 정책(exploration policy) $\mu'$을 구성합니다.

$$
\mu'(s_t) = \mu(s_t|\theta^{\mu}_t) + \mathcal{N}
$$

$\mathcal{N}$는 환경에 맞게 선택할 수 있습니다.  **Ornstein-Uhlenbeck 프로세스**를 사용하여 시간적으로 상관된 탐색을 생성합니다.

## Import modules

## 환경
*ActionNormalizer*는 (-1.1) 범위의 액션 값을 정규화하는 액션 래퍼 클래스입니다. 이 클래스 덕분에 에이전트가 0 중심 범위(-1, 1) 내에서 단순히 액션 값을 선택하도록 할 수 있습니다.

You can see [the code](https://github.com/openai/gym/blob/master/gym/envs/classic_control/pendulum.py) and [configurations](https://github.com/openai/gym/blob/cedecb35e3428985fd4efad738befeb75b9077f1/gym/envs/__init__.py#L81) of Pendulum-v0 from OpenAI's repository.


| Num | Action | Min  | Max |
|-----|--------|------|-----|
| 0   | Torque | -2.0 | 2.0 |

| Num | Observation      | Min  | Max |
|-----|------------------|------|-----|
| 0   | x = cos(theta)   | -1.0 | 1.0 |
| 1   | y = sin(angle)   | -1.0 | 1.0 |
| 2   | Angular Velocity | -8.0 | 8.0 |

r = -(theta<sup>2</sup> + 0.1 * theta_dt<sup>2</sup> + 0.001 * torque<sup>2</sup>)

최소 reward : -(pi<sup>2</sup> + 0.1 * 8<sup>2</sup> + 0.001 * 2<sup>2</sup>) = -16.2736044  
최대 reward : 0 (수직 위치)

### Replay Buffer

## OU Noise
- Exploitation/Exlporation Dilemma 해결책으로 neural network 에서 출력한 action 에 약간의 noise 를 추가  
- DDPG 가 off-policy 알고리즘이므로 noise 를 추가하는 policy 에서 선택된 action 으로 생성된 sample 도 사용 가능  
**Ornstein-Uhlenbeck (확률미분방정식)** 프로세스는 시간 상관 탐색을 생성하고 관성의 물리적 제어 문제에 효과적으로 대처합니다.

$$
dx_t = \theta(\mu - x_t) dt + \sigma dW_t
$$

## Network
Actor와 critic에 대해 두 개의 별도 네트워크를 사용할 것입니다. Actor network의 출력층에는 **tanh**가 사용됩니다. 반면 Critic 네트워크의 출력층은 linear 입니다. 또한 Critic 네트워크의 입력 크기는 state 크기와 action 크기의 합입니다. 

## DDPG Agent
다음은 DDPGAgent 클래스에 대한 요약입니다.

| Method           | Note                                                 |
|---               |---                                                  |
|select_action     | select an action from the input state.               |
|update_model      | update the model by gradient descent.                |
|train             | train the agent during num_frames.                   |
|\_target_soft_update| soft update from the local model to the target model.|
|\_plot              | plot the training progresses.                        |

## Initialize

## Train

## Test
훈련된 에이전트를 실행합니다(1개 에피소드).

## Render